In [9]:
import math

from projectq import MainEngine, ops
from projectq.ops import H, Z, X, Measure, All, CNOT, Rz
from projectq.meta import Loop, Compute, Uncompute, Control, Dagger

from qcircuit import QCircuit, Classic, Qubit, Qureg

In [2]:
init_bell_pair = QCircuit.create(b1=Qubit, b2=Qubit)

def create_bell_pair(eng):
    b1 = eng.allocate_qubit()
    b2 = eng.allocate_qubit()
    
    init_bell_pair(eng, b1, b2)
    
    return b1, b2

In [3]:
create_state = QCircuit.create(psi=Qubit)

In [4]:
restore_state = QCircuit.create(b1=Qubit, b2=Qubit, psi=Qubit)        

def restore_state(eng, b1, b2, psi):
    # Bob may have to apply up to two operation depending on the message sent
    # by Alice:
    with Control(eng, b1):
        X | b2
    with Control(eng, psi):
        Z | b2



In [7]:
def run_teleport(eng, state_creation_function, verbose=False):
    # make a Bell-pair
    b1, b2 = create_bell_pair(eng)

    # Alice creates a nice state to send
    psi = eng.allocate_qubit()
    if verbose:
        print("Alice is creating her state from scratch, i.e., |0>.")
    state_creation_function(eng, psi)

    # entangle it with Alice's b1
    
    with Control(eng, psi):
        X | b1
        
    if verbose:
        print("Alice entangled her qubit with her share of the Bell-pair.")

    # measure two values (once in Hadamard basis) and send the bits to Bob
    H | psi
    Measure | psi
    Measure | b1
    msg_to_bob = [int(psi), int(b1)]
    if verbose:
        print("Alice is sending the message {} to Bob.".format(msg_to_bob))

    restore_state(eng, b1, b2, psi)
    
    # try to uncompute the psi state
    if verbose:
        print("Bob is trying to uncompute the state.")
    with Dagger(eng):
        state_creation_function(eng, b2)

    # check whether the uncompute was successful. The simulator only allows to
    # delete qubits which are in a computational basis state.
    del b2
    eng.flush()

    if verbose:
        print("Bob successfully arrived at |0>")


if __name__ == "__main__":
    eng = MainEngine()
    run_teleport(eng, create_state, verbose=True)

(Note: This is the (slow) Python simulator.)
Debug:  H  |  b1
{'b1': [<projectq.types._qubit.Qubit object at 0x000001F308EBD1D0>], 'b2': [<projectq.types._qubit.Qubit object at 0x000001F308EBD278>]} b1 Qureg[0]
Debug:  X  with Control( ['0'] ) Qureg[0]


AttributeError: 'Qureg' object has no attribute 'id'
 raised in:
'  File "C:\\Users\\kotko\\AppData\\Roaming\\Python\\Python37\\site-packages\\projectq\\ops\\_command.py", line 241, in <listcomp>'
'    for qubit in qubits])'

In [80]:
eng = MainEngine()

###



import importlib
mod = importlib.import_module('projectq.ops')
arbitraryOp = getattr(mod, 'Rx')



###

psi = eng.allocate_qubit()
arbitraryOp(3.14 / 2) | psi
mod.Measure | psi
print(int(psi))

(Note: This is the (slow) Python simulator.)
0
